In [1]:
import sys
sys.path.append('../..')
from src.pomdp import *

import pandas as pd
import numpy as np
import cupy as cp

In [2]:
# For my computer: so it doesn't use shared GPU memory which is slower
mempool = cp.get_default_memory_pool()
mempool.set_limit(size=3.7*1024**3) # Allowing cupy 3.8 GB of GPU memory

## Model definition

In [3]:
olfactory_data = pd.read_csv('./Data/statistics_nose_3e6_35_153.dat', sep=',', header=None).values
olfactory_data = np.array(olfactory_data)

: 

In [ ]:
# Model size
env_shape = olfactory_data.shape
state_count = env_shape[0] * env_shape[1]

print(f'Environment shape: {env_shape}')
print(f'State count: {state_count}')

In [ ]:
# goal state
olfactory_data[17,0]

In [ ]:
def id_from_pos(x,y):
    return (x * env_shape[1]) + y

In [ ]:
# states
state_grid = []

# observation matrix
flattened_olfactory_data = olfactory_data.flatten()

observations = np.zeros((state_count, 4, 2))
observations[:,:,0] = flattened_olfactory_data.reshape((flattened_olfactory_data.shape[0],1)).repeat(4, axis=1)
observations[:,:,1] = (1 - observations[:,:,0])

# State labels
for i in range(env_shape[0]):
    i_states = []
    for j in range(env_shape[1]):
        i_states.append(f's_{i}_{j}')

    state_grid.append(i_states)